In [1]:
import numpy as np
from sklearn.model_selection import cross_val_score
import statistics as stat
from sklearn.preprocessing import StandardScaler

In [2]:
def dropout(x: np.ndarray, m: int, ϕ: float) -> np.ndarray:
    """ Regularizes matrix x using dropout """
    
    assert(0 <= ϕ <= 1)  # r is a valid probability
    
    # perform dropout w/ bernoulli dist.
    _, p  = x.shape
    drop  = np.random.binomial(1, 1-ϕ, (m, p))
    x_out = np.concatenate([d*x for d in drop])
    
    return x_out

In [3]:
def add_random_noise(x, sd=0.5):
    """Adds a independent random value(from normal(0,sd)) to each element of matrix X
    """
    return x + np.random.normal(loc = 0, scale=sd, size=x.shape)

In [4]:
def gen_delta_grid(X, c, n):
    """ Returns list of n random matrices such the the l2 norm of the jth colum is equal to c[j]
    """
    dim = X.shape
    matrix_grid = []
    for _ in range(n):
        # generate random nxp matrix
        rand_mat = np.random.randn(*dim)
        # append list with matrix with normalized columns = cj 
        matrix_grid.append((rand_mat / np.sqrt(np.square(rand_mat).sum(axis=0))[None, :])*c[None, :])
    return matrix_grid

In [5]:
def HW3 (learning_alg, X_train, Y_train, regularization, M, c, K, Error_Type):
    """Automatically tunes a Blackbox regression model.

    Parameters
    ----------
    learning_alg : function
        Takes as input an matrix X ∈ R n×p and a vector of responses Y ∈ R n 
        and returns a function that maps inputs to outputs.
        
    X_train : array-like of shape (n,p)
        An array of training data.
        
    Y_train : array-like of shape (n,)
        A vector of response values to X_train.
        
    regularization : str
        A regularization method that belongs to the set
        {Dropout, NoiseAddition, Robust}.
        
    M : int
        A positive integer that specifies the number of Monte Carlo replicates
        to be used if the regularization specified is Dropout or NoiseAddition.

    c : array-like of shape (p,)
        A vector of column bounds to be used if the method specified
        is Robust.
        
    K : int
        A positive integer indicating the number of CV-folds to be
        used to tune the amount of regularization.

    Error_Type : str
        A criterion to be used to evaluate the regularization that belongs to
        the set {MSE, MAD} where MSE encodes mean square error and
        MAD encodes mean absolute deviation.

    Returns
    ------- 
    final_model: function
        A predictive model that optimizes the specified criterion using 
        the specified regularization method.
    """
    #Standardize the Data
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    
    #Chooses between MSE and MAD based on input
    if Error_Type == 'MSE':
        score_type = 'neg_mean_squared_error'
    else:
        score_type = 'neg_mean_absolute_error'
    
    #Dropout Regularization##################################
    if regularization == 'Dropout':
        
        #Create dropout probabilities to test
        ϕ = np.linspace(0,1)
        
        #Sets base cases for optimization
        best_ϕ = 0
        X_new = dropout(X_train,M,best_ϕ)
        Y_new = np.tile(Y_train,M)
        best_error = stat.mean(cross_val_score(learning_alg,X_new,Y_new,cv = K,scoring = score_type))
        
        #Tests the CV error of each ϕ value and saves the best error and corresponding ϕ
        for i in ϕ:
            X_new = dropout(X_train,M,i)
            error = stat.mean(cross_val_score(learning_alg,X_new,Y_new,cv = K,scoring = score_type))
            
            if error <= best_error:
                best_error = error
                best_ϕ = i
        
        #Creates the optimal X and then fits the model on it
        X_best = dropout(X_train,M,best_ϕ)
        learning_alg_final = learning_alg.fit(X_best,Y_new)
        
        #The function to return
        def final_model(X):
            
            scaler = StandardScaler()
            X= scaler.fit_transform(X)
            X_final = dropout(X,best_ϕ,M) 
            return learning_alg_final.predict(X_final)
        
        #Returning the final dropout model
        return final_model

    #Noise Addition Regularization##########################
    elif regularization == 'NoiseAddition':
        
        #Create dropout probabilities to test
        σ = np.linspace(0,5,11)
        
        #Sets base cases for optimization
        best_σ = 0
        best_error = stat.mean(cross_val_score(learning_alg,X_train,Y_train,cv = K,scoring = score_type))

        #Tests the CV error of each σ value and saves the best error and corresponding σ
        for i in σ:
            X_new = add_random_noise(X_train,i)
            error = stat.mean(cross_val_score(learning_alg,X_new,Y_train,cv = K,scoring = score_type))
            
            if error <= best_error:
                best_error = error
                best_σ = i
        
        #Creates the optimal X and then fits the model on it
        X_best = add_random_noise(X_train,best_σ)
        learning_alg_final = learning_alg.fit(X_best,Y_train)
        
        #The function to return
        def final_model(X):
            scaler = StandardScaler()
            X= scaler.fit_transform(X)
            X_final = add_random_noise(X,best_σ) 
            return learning_alg_final.predict(X_final)
        
        return learning_alg.train(X_best,Y_new)
     
    #Robust Regularization##################################
    else:
        
        #this function returns a list of matrices
        δ_mats = function(X_train,c,M) 
        
        #Sets base cases for optimization
        best_δ = δ_mats[0]
        best_error = stat.mean(cross_val_score(learning_alg,X_train+best_δ,Y_train,cv = K,scoring = score_type))
        
        #Tests the CV error of each δ matrix and saves the best error and corresponding δ
        for δ in δ_mats:
            new_X = X_train+δ
            error = stat.mean(cross_val_score(learning_alg,X_new,Y_train,cv = K,scoring = score_type))
            
            if error <= best_error:
                best_error = error
                best_δ = δ
        
        return learning_alg.train(X_train+δ,Y_train)


In [6]:
np.linspace(0,5,11)

array([0. , 0.5, 1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5, 5. ])